<img width="10%" alt="Naas" src="https://landen.imgix.net/jtci2pxwjczr/assets/5ice39g4.png?w=160"/>

# Google Sheets - Calculate distance and price
<a href="https://app.naas.ai/user-redirect/naas/downloader?url=https://raw.githubusercontent.com/jupyter-naas/awesome-notebooks/master/Google%20Sheets/Google_Sheets_Get_data.ipynb" target="_parent"><img src="https://naasai-public.s3.eu-west-3.amazonaws.com/Open_in_Naas_Lab.svg"/></a><br><br><a href="https://bit.ly/3JyWIk6">Give Feedbacks</a> | <a href="https://app.naas.ai/user-redirect/naas/downloader?url=https://raw.githubusercontent.com/jupyter-naas/awesome-notebooks/master/Naas/Naas_Start_data_product.ipynb" target="_parent">Generate Data Product</a>

**Tags:** #googlesheets #gsheet #data #naas_drivers #operations #snippet #dataframe #google_maps_api #routes_api

**Author:** [Antonio Georgiev](www.linkedin.com/in/antonio-georgiev-b672a325b)

**Description:** This template determines the cost and distance between location extracted from a Google Sheet. It uses the Routes API to estimate the price depending on the distance between sites and outputs the updated Google Sheet with distances and prices.

## Input

### Import library

In [1]:
import naas
from naas_drivers import gsheet
import requests

### Setup Google Sheets
- Share your Google Sheets spreadsheet with our service account : 🔗 naas-share@naas-gsheets.iam.gserviceaccount.com

In [2]:
SPREADSHEET_URL = (
    "https://docs.google.com/spreadsheets/d/1uC35Se4LtHUSQm8EUypAm6psH_q62mQJtLKDSSaKBdI/edit?usp=sharing"
)
SHEET_NAME = "Random"

### Setup API

In [3]:
api_secret_name = "API" # add your API key directly in the API_KEY variable or using naas secret variable
API_KEY = naas.secret.get(api_secret_name)  # Read API key from the secret variable
API_ENDPOINT = 'https://routes.googleapis.com/directions/v2:computeRoutes'
GEOCODING_API_ENDPOINT = "https://maps.googleapis.com/maps/api/geocode/json"

## Model

### Get data from Google Sheets spreadsheet

In [4]:
sheet = gsheet.connect(SPREADSHEET_URL)
df = sheet.get(sheet_name=SHEET_NAME)

### Convert addresses to coordinates

In [5]:
def get_coordinates(address):
    params = {"address": address, "key": API_KEY}
    response = requests.get(GEOCODING_API_ENDPOINT, params=params)
    data = response.json()
    if data["status"] == "OK":
        result = data["results"][0]
        lat = result["geometry"]["location"]["lat"]
        lng = result["geometry"]["location"]["lng"]
        return lat, lng
    else:
        return None

### Update the Google Sheet with the relevant coordinates

In [37]:
for index, row in df.iterrows():
    origin_address = row["Origin"]
    destination_address = row["Destination"]
    
    origin_coordinates = get_coordinates(origin_address)
    if origin_coordinates:
        df.at[index, "Origin lat"] = origin_coordinates[0]
        df.at[index, "Origin lng"] = origin_coordinates[1]
    
    destination_coordinates = get_coordinates(destination_address)
    if destination_coordinates:
        df.at[index, "Destination lat"] = destination_coordinates[0]
        df.at[index, "Destination lng"] = destination_coordinates[1]

### Retrieve distance and toll information from Routes API and update the sheet

In [40]:
headers = {
    'Content-Type': 'application/json',
    'X-Goog-Api-Key': API_KEY,
    'X-Goog-FieldMask': 'routes.duration,routes.distanceMeters,routes.travelAdvisory.tollInfo,routes.legs.travelAdvisory.tollInfo'
}

for index, row in df.iterrows():
    origin_lat = row["Origin lat"]
    origin_lng = row["Origin lng"]
    destination_lat = row["Destination lat"]
    destination_lng = row["Destination lng"]

    payload = {
        "origin": {
            "location": {
                "latLng": {
                    "latitude": origin_lat,
                    "longitude": origin_lng
                }
            }
        },
        "destination": {
            "location": {
                "latLng": {
                    "latitude": destination_lat,
                    "longitude": destination_lng
                }
            }
        },
        "travelMode": "DRIVE",
        "extraComputations": ["TOLLS"],
        "routeModifiers": {
            "vehicleInfo": {
                "emissionType": "GASOLINE"
            },
            "tollPasses": [
               # Add relevant toll passes if applicable
            ]
        }
    }

    response = requests.post(API_ENDPOINT, json=payload, headers=headers)
    if response.status_code == 200:
        data = response.json()
        if "routes" in data and len(data["routes"]) > 0:
            route = data["routes"][0]
            distance_meters = route.get("distanceMeters")
            if distance_meters:
                distance_km = distance_meters / 1000
                df.loc[index, "Distance"] = f"{distance_km:.2f} km"
            toll_info = route.get("travelAdvisory", {}).get("tollInfo", {})
            toll_info = route.get("travelAdvisory", {}).get("tollInfo", {})
            if toll_info:
                estimated_price = toll_info.get("estimatedPrice", [])
                if estimated_price:
                    currency_code = estimated_price[0].get("currencyCode", "")
                    toll_cost = float(estimated_price[0].get("units", 0))
                    formatted_toll_cost = '{:.2f}'.format(toll_cost)
                    df.loc[index, "Toll costs"] = currency_code + ' ' + formatted_toll_cost
                else:
                    df.loc[index, "Toll costs"] = "Not available"
            else:
                df.loc[index, "Toll costs"] = "Not available"
        else:
            print(f"No routes found in the response for row {index+2}")
    else:
        print(f"Route calculation request failed for row {index+2} with status code: {response.status_code}")

## Output

### Save the updated Google Sheet

In [41]:
result = gsheet.connect(SPREADSHEET_URL).send(sheet_name=SHEET_NAME, data=df, append=False)
if "error" in result:
    print("Failed to save the sheet.")
else:
    print("Saved successfully.")

Saved successfully.
